In [1]:
import torch
from yo_fluq_ds import *
from tg.common.ml import batched_training as bt
from tg.grammar_ru.common import Loc, DataBundle
from tg.common.ml.batched_training import Batcher
from tg.grammar_ru.ml.features import PyMorphyFeaturizer
from tg.common.ml.batched_training import IndexedDataBundle
from tg.common.ml.batched_training import train_display_test_split
from tg.grammar_ru.ml.components.core_extractor.extractor import CoreExtractor
from tg.grammar_ru.ml.components.plain_context_builder import PlainContextBuilder
from tg.grammar_ru.ml.components.contextual_binding import ContextualBinding, ContextualNetworkType

Сначала копипаста прошлой части:

In [96]:
db = DataBundle.load(Loc.bundles_path/'antcd/wwd')
db.index

,pronoun_word_id,pronoun_sentence_id,candidate_word_id,candidate_sentence_id,candidate_distance,is_match,word_distance
sample_id,,,,,,,
0,7,0,4,0,1,1,3
1,7,0,5,0,0,0,2
2,514887,514880,514884,514880,1,1,3
3,514887,514880,514885,514880,0,0,2
4,1903357,1903337,1903342,1903335,3,0,12
...,...,...,...,...,...,...,...
101060,22837552,22837430,22837513,22837429,4,0,33
101061,22837552,22837430,22837524,22837430,3,0,25
101062,22837552,22837430,22837537,22837430,2,0,14


In [97]:
db.index = db.index.iloc[:1000]
db.index = db.index.rename(columns={'is_match': 'label'})
db.index['split'] = train_display_test_split(db.index)
db.index

,pronoun_word_id,pronoun_sentence_id,candidate_word_id,candidate_sentence_id,candidate_distance,label,word_distance,split
sample_id,,,,,,,,
0,7,0,4,0,1,1,3,display
1,7,0,5,0,0,0,2,train
2,514887,514880,514884,514880,1,1,3,test
3,514887,514880,514885,514880,0,0,2,train
4,1903357,1903337,1903342,1903335,3,0,12,test
...,...,...,...,...,...,...,...,...
995,2011233,2010938,2011213,2010935,2,0,14,train
996,2011233,2010938,2011223,2010937,1,0,7,test
997,2011233,2010938,2011227,2010937,0,1,4,test


In [98]:
idb = IndexedDataBundle(db.index, db)
idb

In [6]:
from tg.grammar_ru.ml.components.training_task_factory import TaskFactory, Conventions

def get_binary_label_extractor():
    label_extractor = (bt.PlainExtractor
                       .build(Conventions.LabelFrame)
                       .index()
                       .apply(take_columns=['label'], transformer=None))
    return label_extractor

def get_plain_context(sentence_id_column_name, word_id_column_name):
    pcb = PlainContextBuilder(include_zero_offset=True, left_to_right_contexts_proportion=0.5)
    pcb.sentence_id_column_name = sentence_id_column_name
    pcb.word_id_column_name = word_id_column_name
    plain_context = ContextualBinding(
        name='plain_context',
        context_length=3,
        network_type=ContextualNetworkType.Plain,
        hidden_size=[30],
        context_builder=pcb,
        extractor=CoreExtractor(join_column='another_word_id'),
        debug=False
    )
    return plain_context

Для каждой головы свой батч с контекстом нужных слов

In [7]:
pronoun_pc = get_plain_context('pronoun_sentence_id', 'pronoun_word_id')
pronoun_head_factory = pronoun_pc.create_network_factory(task=None, input=None)
candidate_pc = get_plain_context('candidate_sentence_id', 'candidate_word_id')
candidate_head_factory = candidate_pc.create_network_factory(task=None, input=None)

context_extractors = [pronoun_pc.create_extractor(task=None, bundle=idb),
                      candidate_pc.create_extractor(task=None, bundle=idb)]
for extractor in context_extractors:
    extractor.fit(idb)
batches = [Batcher(batch_size=1000, extractors=[extractor, get_binary_label_extractor()]).fit_extract(idb) for extractor in context_extractors]

2022-12-06 11:59:13.078511+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-12-06 11:59:13.319052+00:00 INFO: Success
2022-12-06 11:59:13.319052+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-12-06 11:59:13.569993+00:00 INFO: Success
2022-12-06 11:59:13.569993+00:00 INFO: Fitting extractor slovnet_syntax in CoreExtractor
2022-12-06 11:59:13.788421+00:00 INFO: Success
2022-12-06 11:59:13.788421+00:00 INFO: Fitting extractor syntax_fixes in CoreExtractor
2022-12-06 11:59:13.804048+00:00 INFO: Skipped as the corresponding frame is missing from the bundle
2022-12-06 11:59:13.804048+00:00 INFO: Fitting extractor syntax_stats in CoreExtractor
2022-12-06 11:59:13.804048+00:00 INFO: Skipped as the corresponding frame is missing from the bundle
2022-12-06 11:59:15.295277+00:00 INFO: Fitting extractor pymorphy in CoreExtractor
2022-12-06 11:59:15.534832+00:00 INFO: Success
2022-12-06 11:59:15.534832+00:00 INFO: Fitting extractor slovnet_morph in CoreExtractor
2022-

Попытка соединить головы:

In [70]:
from tg.common.ml.batched_training.torch.networks.simple_networks import FullyConnectedNetwork
from tg.common.ml.batched_training.torch.networks.network_commons import TorchNetworkFactory
from tg.common.ml.batched_training.torch.networks.extracting_network import FeedForwardNetwork

class ParallelNetworkDifferentInputs(torch.nn.Module):
    def __init__(self, *networks: torch.nn.Module):
        super(ParallelNetworkDifferentInputs, self).__init__()
        self.networks = networks[0]

    def forward(self, inputs):
        result = []
        for i in range(len(self.networks)):
            network = self.networks[i]
            result.append(network(inputs[i]))
        return result

class HeadsUnionNetwork(torch.nn.Module):
    def __init__(self,
                 sizes: List[int],
                 heads,
                 outs,
                 output: Union[None, torch.Tensor, int] = None):
        super(HeadsUnionNetwork, self).__init__()
        heads_forward_networks = []
        for i in range(len(heads)):
            fully_connected = FullyConnectedNetwork(sizes=[3], input=outs[i].shape[1], output=output)
            heads_forward_networks.append(FeedForwardNetwork(heads[i], fully_connected, torch.nn.Softmax(dim=1)))
        #self.parallel_network = ParallelNetworkDifferentInputs(heads_forward_networks)
        self.nn_layers = torch.nn.ModuleList([ParallelNetworkDifferentInputs(heads_forward_networks)])

    def forward(self, inputs):
        for i, layer in enumerate(self.nn_layers):
            heads_outputs = layer(inputs) #[x['plain_context'] for x in inputs]
        concat = torch.cat([torch.transpose(x, 0, 1) for x in heads_outputs], dim=0)
        parallel_mean = torch.mean(concat, 0)
        return parallel_mean.view(len(parallel_mean), 1)
    

class MultiheadNetworkFactory:
    def __init__(self, *head_factories):
        self.head_factories = [*head_factories]

    def create_network(self, task, inputs):
        if len(self.head_factories) != len(inputs):
            raise Error
        heads = []
        outs = []
        for i in range(len(inputs)):
            head = self.head_factories[i].create_network(task=None, input=inputs[i])
            heads.append(head)
            outs.append(head(inputs[i]))

        return HeadsUnionNetwork(sizes=[3], heads=heads, outs=outs, output=inputs[0]['label'].shape[1])

In [71]:
network_factory = MultiheadNetworkFactory(pronoun_head_factory, candidate_head_factory)
assembled_network = network_factory.create_network(task=None, inputs=batches)

Теряются параметры

In [76]:
def train_loop(model, loss_fn, batches):
    pred = model(batches)
    loss = loss_fn(pred, torch.tensor(batches[0]['label'].values, dtype=torch.float32))

    #optimizer.zero_grad()
    loss.backward()
    #optimizer.step()
    
learning_rate = 1
#optimizer = torch.optim.SGD(assembled_network.parameters(), lr=learning_rate)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(assembled_network, torch.nn.CrossEntropyLoss(), batches)
print("Done!")

Epoch 1
-------------------------------
Epoch 2
-------------------------------
Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------
Epoch 6
-------------------------------
Epoch 7
-------------------------------
Epoch 8
-------------------------------
Epoch 9
-------------------------------
Epoch 10
-------------------------------
Done!


In [94]:
optimizer = torch.optim.SGD(assembled_network.parameters(), lr=learning_rate)

ValueError: optimizer got an empty parameter list

Выглядит плохо

Смотрю на tasks

In [91]:
from tg.common.ml.batched_training import torch as btt
from tg.common.ml.batched_training import TrainingSettings
from tg.common.ml import batched_training as bt
    
class MultiheadExtractorsFactory(btt.TorchExtractorFactory):
        def __init__(self, plain_contexts):
            self.plain_contexts = plain_contexts

        def create_extractors(self, task, bundle) -> List[bt.Extractor]:
            context_extractors = []
            for pc in self.plain_contexts:
                context_extractors.append(pc.create_extractor(task=task, bundle=bundle))
            return context_extractors

class MyTrainingTask(btt.TorchTrainingTask):
    def __init__(self,
                 training_settings: TrainingSettings,
                 torch_settings: btt.TorchTrainingSettings,
                 metric_pool: bt.MetricPool,
                 basis_tasks_sources: Optional[Dict[str, btt.AbstractBasisTaskSource]] = None
                 ):
        plain_contexts = [get_plain_context('pronoun_sentence_id', 'pronoun_word_id'),
                          get_plain_context('candidate_sentence_id', 'candidate_word_id')]
        head_factories = [pc.create_network_factory(task=None, input=None) for pc in plain_contexts]
        
        extractor_factory = MultiheadExtractorsFactory(plain_contexts)
        network_factory = MultiheadNetworkFactory(*head_factories)
        super(MyTrainingTask, self).__init__(
            training_settings,
            torch_settings,
            extractor_factory,
            network_factory,
            metric_pool,
            basis_tasks_sources
        )

In [92]:
import typing as tp
from pathlib import Path

from sklearn.metrics import roc_auc_score
from tg.grammar_ru.ml.components import GrammarMirrorSettings
from tg.common.ml import batched_training as bt



features = {
    'P': 'pymorphy',
    'M': 'slovnet_morph',
    'S': 'slovnet_syntax',
    'F': 'syntax_fixes',
    'T': 'syntax_stats'
}


def build_task(
        epoch_count: int = 50,
        batch_size: int = 20000,
        mini_batch_size: int = 200,
        mini_epoch_count: int = 4,
        learning_rate: float = 0.1,
        plain_context_length: int = 10,
        plain_context_left_shift: float = 0.5,
        plain_net_size: tp.List[int] = [20],
        plain_network_mode: btm.ContextualNetworkType = btm.ContextualNetworkType.Plain,
        plain_context_reverse: bool = False,
        feature_allow_list: tp.Optional[tp.List[tp.Any]] = None
        ) -> btm.MirrorTrainingTask:
    train_settings = bt.TrainingSettings(
        epoch_count=epoch_count,
        batch_size=batch_size,
        mini_batch_size=mini_batch_size,
        mini_epoch_count=mini_epoch_count
    )
    torch_settings = btt.TorchTrainingSettings(
        btt.OptimizerConstructor('torch.optim:SGD', lr=learning_rate)
    )
    mirror_settings = GrammarMirrorSettings()
    mirror_settings.plain_context.context_builder.left_to_right_contexts_proportion = plain_context_left_shift
    mirror_settings.plain_context.context_length = plain_context_length
    mirror_settings.plain_context.hidden_size = plain_net_size
    mirror_settings.plain_context.network_type = plain_network_mode
    mirror_settings.plain_context.reverse_order_in_lstm = plain_context_reverse

    if feature_allow_list is not None:
        mirror_settings.plain_context.extractor.allow_list = [features[_] for _ in feature_allow_list]

    task = MyTrainingTask(
        train_settings,
        torch_settings,
        bt.MetricPool().add_sklearn(roc_auc_score),
    )
    task.info['name'] = 'TSAG-'
    return task


def run_local(bundle: DataBundle) -> None:
    task = build_task(plain_network_mode=btm.ContextualNetworkType.Plain)
    task.settings.epoch_count = 10
    task.settings.batch_size = 1000
    task.settings.training_batch_limit = 10
    task.settings.evaluation_batch_limit = None

    return (task, task.run(bundle))

In [99]:
run_local(idb)

2022-12-06 18:39:25.849362+00:00 INFO: Training starts. Info: {'name': 'TSAG-'}
2022-12-06 18:39:25.851361+00:00 INFO: Ensuring/loading bundle. Bundle before:
2022-12-06 18:39:25.853362+00:00 INFO: Bundle loaded
{'index': {'shape': (101065, 7), 'index_name': 'sample_id', 'columns': ['pronoun_word_id', 'pronoun_sentence_id', 'candidate_word_id', 'candidate_sentence_id', 'candidate_distance', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'pymorphy': {'shape': (570787, 16), 'index_name': 'word_id', 'columns': ['normal_form', 'alternatives', 'score', 'delta_score', 'POS', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'slovnet': {'shape': (570787, 17), 'index_name': 'word_id', 'columns': ['POS', 'Case', 'Animacy', 'Gender', 'Number', '...'], 'index': [0, 1, 2, 3, 4, '...']}, 'src': {'shape': (570787, 14), 'index_name': None, 'columns': ['word_id', 'sentence_id', 'word_index', 'paragraph_id', 'word_tail', '...'], 'index': [0, 1, 2, 3, 4, '...']}}
2022-12-06 18:39:25.855363+00:00 INFO: Index frame is

MemoryError: cannot allocate memory for array